# ADM Quiz

**Instructions**

**Make a copy of this notebook.**

**Rename the notebook to *your_name - ADM Quiz*.**

There are 8 questions in this quiz. Please attempt all questions. You may refer to the course materials, the internet and any resources you encounter. Discussions are welcomed, but you have to submit an individual answer notebook.

**Q1.** Load the two csvs *privateprices-sep2018.csv* and *privateprices-feb2019.csv* to dataframes **df_092018** and **df_022019**

Print the info() of both dataframes.



In [1]:
from google.colab import drive

# mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
from pathlib import Path

# To check what is the directory listing, we use the built-in function .listdir
os.listdir('/content/drive')

# The data we want to load is stored in '/content/drive/data/ADM'
folder_path = Path('/content/drive/My Drive/pcml_data/4ADM')

# Write your answer for Q1 here
df_092018 = pd.read_csv(folder_path / 'privateprices-sep2018.csv')
df_022019 = pd.read_csv(folder_path / 'privateprices-feb2019.csv')

print(df_092018.info())
print(df_022019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Project Name                 1000 non-null   object
 1   Address                      1000 non-null   object
 2   No. of Units                 1000 non-null   int64 
 3   Area (sqm)                   1000 non-null   int64 
 4   Type of Area                 1000 non-null   object
 5   Transacted Price ($)         1000 non-null   int64 
 6   Nett Price($)                1000 non-null   object
 7   Unit Price ($ psm)           1000 non-null   int64 
 8   Unit Price ($ psf)           1000 non-null   int64 
 9   Sale Date                    1000 non-null   object
 10  Property Type                1000 non-null   object
 11  Tenure                       1000 non-null   object
 12  Completion Date              1000 non-null   object
 13  Type of Sale                 1000 

**Q2.** Plot the probability distribution function (PDF) for both dataframes on *'Transacted Price ($)'* with nbins=50, histnorm=density.

**What kind of distribution do you think best fits the Transacted Price?**
(You can write as comments in the code block.)


In [3]:
import plotly.express as px

# Write your answer for Q2 here
fig_092018 = px.histogram(x=df_092018['Transacted Price ($)'], nbins=50, histnorm='density')
fig_092018.show()

fig_022019 = px.histogram(x=df_022019['Transacted Price ($)'], nbins=50, histnorm='density')
fig_022019.show()

# A poisson distribution best fits the Transacted Price

In [4]:
import numpy as np

# normal distribution
prop_norm = np.random.normal(df_092018['Transacted Price ($)'].mean(), df_092018['Transacted Price ($)'].std(), size=10000)
fig_norm = px.histogram(x=prop_norm, nbins=50, histnorm='density')
fig_norm.show()

In [5]:
# poisson distribution
prop_poi = np.random.poisson(1.5, size=10000)
fig_poi = px.histogram(x=prop_poi, nbins=50, histnorm='density')
fig_poi.show()

In [6]:
# exponential distribution
prop_exp = np.random.exponential(1/df_092018['Transacted Price ($)'].mean(), size=10000)
fig_exp = px.histogram(x=prop_exp, nbins=50, histnorm='density')
fig_exp.show()

**Q3.** Plot the Empirical Cumulative Distribution Function (ECDF) for both dataframes' Transacted Price in two different charts.


In [7]:
import numpy as np
# Write your answer for Q3 here

def ecdf(data):
    """Compute ECDF and generate a dataframe."""
    df = pd.DataFrame()
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    # sort the data
    df['x'] = np.sort(data)

    # y-data for the ECDF: y
    # y goes from 1/n to 1 in equal spaced intervals, arange() does that
    # but arange end value is not inclusive, so n+1
    df['y'] = np.arange(1 , n+1)  / n

    return df

fig_092018 = px.scatter(
    ecdf(df_092018['Transacted Price ($)']), x='x', y='y', labels={'x': 'Transacted Price', 'y': 'ECDF'},
    opacity=0.5)
fig_092018.show()

fig_022019 = px.scatter(
    ecdf(df_022019['Transacted Price ($)']), x='x', y='y', labels={'x': 'Transacted Price', 'y': 'ECDF'},
    opacity=0.5)
fig_022019.show()

**Q4.**
Create a list that contains 1000 bootstrapped replicates (each bootstrap sample of 1000 observations) of the mean of df_022019 Transacted Price.

Find the two-tailed 95% confidence interval.

In [8]:
# Write your answer for Q4 here

def bs_replicate(data, func, nsize=1):
  """ Gives bootstrap replicate of data."""
  bs_sample = np.random.choice(data, size=nsize)
  return func(bs_sample)

# make function that loops through bs_replicate()
def bs_replicate_reps(data, func, nsize=1, n=1):
  """Loop over bs_replicate function."""
  bs_replicates = np.empty(n)

  for i in range(n):
    bs_replicates[i] = bs_replicate(data, func, nsize)
  return bs_replicates

df_0220199_reps = bs_replicate_reps(df_022019['Transacted Price ($)'], np.mean, nsize=1000, n=1000)

df_022019_conf_int_95 = np.percentile(df_0220199_reps, [2.5, 97.5])
print('The two-tailed 95% confidence interval for the 2019 Transacted Price bootstrap sample means is', df_022019_conf_int_95)

The two-tailed 95% confidence interval for the 2019 Transacted Price bootstrap sample means is [1633596.93365 1849686.8862 ]


**Q5.**
Calculate the difference in means of df_092018 and df_022019's Transacted Price (022019 - 092018).

Generate 10000 permutation replicates of the difference of means of df_092018 and df_022019's Transacted Price (022019 - 092018).

Find the p-value based on the actual difference in means and the permutation replicate means.

**Can we reject the hypothesis that the means are the same? based on an $\alpha$ of 5%.** (You can write as comments in the code block.)

In [9]:
# Write your answer for Q5 here

def perm_sample(data1, data2):
  """Gives a permutation sample from two arrays data1 and data2."""

  # use pd.concat to combine the two datasets columns
  comb_data = np.concatenate((data1, data2))

  # permutate the comb_data
  perm_data = np.random.permutation(comb_data)

  # split up the perm_data into two
  perm_sample1  = perm_data[:len(data1)]
  perm_sample2  = perm_data[len(data1):]

  return perm_sample1, perm_sample2

# create function to generate permutation replicates

def perm_reps(data_1, data_2, func, size=1):
    """Gives multiple permutation replicates."""

    # create empty array
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample1, perm_sample2 = perm_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample1, perm_sample2)

    return perm_replicates

# Function that calculate the difference between means

def diff_means(data1, data2):
  """Calculate difference between means."""
  difference = np.mean(data1) - np.mean(data2)

  return difference

diff_092018_022019 = diff_means(
    df_022019['Transacted Price ($)'],
    df_092018['Transacted Price ($)'])
print(f'Difference between actual means: {diff_092018_022019}')

diff_perm_rep_092018_022019 = perm_reps(
    df_022019['Transacted Price ($)'],
    df_092018['Transacted Price ($)'],
    diff_means,
    100)

pvalue = np.sum(diff_perm_rep_092018_022019 >= diff_092018_022019) / len(diff_perm_rep_092018_022019)
print(f'The p-value is {pvalue}')

# Fail to reject the null that means are the same

Difference between actual means: 41858.260469713015
The p-value is 0.45


**Qn 6.**
Using only df_022019, fit a linear regression of '*Transacted Price ($)*' (y dependent variable) on '*Postal District*' and '*Postal Sector*' (x independent variables).

**Look at the summary of the model. Does it tell you anything useful?** (You can write as comments in the code block.)

In [10]:
import statsmodels.api as sm

# Write your answer for Q6 here

y = df_022019['Transacted Price ($)']

x = sm.add_constant(df_022019[['Postal District', 'Postal Sector']])

model = sm.OLS(y, x).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     Transacted Price ($)   R-squared:                       0.033
Model:                              OLS   Adj. R-squared:                  0.031
Method:                   Least Squares   F-statistic:                     15.96
Date:                  Sat, 17 Jun 2023   Prob (F-statistic):           1.52e-07
Time:                          06:18:27   Log-Likelihood:                -14830.
No. Observations:                   941   AIC:                         2.967e+04
Df Residuals:                       938   BIC:                         2.968e+04
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            2.318

**Qn 7.**
Using df_022019, create a new dataframe dummy_022019 and put in dummy variables using '*Planning Region*'. (Hint: Use pd.get_dummies())

It should look something like this:
![dummy](https://i.imgur.com/XqGkqKf.png)



Add a constant to the dataframe.

Now fit a linear regression of '*Transacted Price ($)*' on the dummy variables.

Look at the model summary.

**Which region has a negative impact on prices?
Is it statistically significant?**

**Which region has the largest impact on prices?
Is it statistically significant?**

(You can write as comments in the code block.)

In [11]:
# Write your answer for Q7 here

df_022019['Planning Region'].unique()

array(['East Region', 'North East Region', 'Central Region',
       'West Region', 'North Region'], dtype=object)

In [12]:
# Write your answer for Q8 here
dummy_022019 = pd.get_dummies(df_022019[['Planning Region']])

x = sm.add_constant(dummy_022019)
x

,const,Planning Region_Central Region,Planning Region_East Region,Planning Region_North East Region,Planning Region_North Region,Planning Region_West Region
0,1.0,0,1,0,0,0
1,1.0,0,0,1,0,0
2,1.0,0,1,0,0,0
3,1.0,1,0,0,0,0
4,1.0,1,0,0,0,0
...,...,...,...,...,...,...
936,1.0,0,1,0,0,0
937,1.0,1,0,0,0,0
938,1.0,1,0,0,0,0
939,1.0,1,0,0,0,0


In [13]:
df_merged2019 = pd.merge(df_022019, dummy_022019, how='left', left_index=True, right_index=True)

y = df_merged2019['Transacted Price ($)']

model = sm.OLS(y, x).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     Transacted Price ($)   R-squared:                       0.058
Model:                              OLS   Adj. R-squared:                  0.054
Method:                   Least Squares   F-statistic:                     14.36
Date:                  Sat, 17 Jun 2023   Prob (F-statistic):           2.20e-11
Time:                          06:18:27   Log-Likelihood:                -14818.
No. Observations:                   941   AIC:                         2.965e+04
Df Residuals:                       936   BIC:                         2.967e+04
Df Model:                             4                                         
Covariance Type:              nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

**Qn 8.** How may you improve the model, using existing variables in the dataset?

(You can write as comments in the code block.)

In [14]:
# Include additional features from the dataset and appropriately encode categorical variables as dummy variables.